Отлично, давай оптимизируем конспект по Логистической Регрессии, включая бинарный и мультиклассовый случаи.

---

### XI.A. Логистическая Регрессия: Основы

**Назначение:** Логистическая регрессия — это **линейный метод для задач классификации**. В отличие от линейной регрессии (предсказывающей непрерывные значения), логистическая регрессия предсказывает **вероятность** принадлежности объекта к определенному классу.

**Проблема Линейной Регрессии для Классификации:**
Линейная регрессия ($y = Xw + b$) выдает значения в диапазоне $(-\infty, +\infty)$, что не подходит для вероятностей (требуется диапазон [0, 1]).

**Решение: Сигмоидальная Функция (Sigmoid / Logistic Function):**
Логистическая регрессия использует сигмоиду для "сжатия" выхода линейной комбинации в диапазон [0, 1].

**Как работает (Бинарный случай):**

1.  **Линейная комбинация:** Вычисляется $z = Xw + b$.
2.  **Сигмоида:** $z$ пропускается через сигмоиду:
    $$\sigma(z) = \frac{1}{1 + e^{-z}}$$
3.  **Выход ($\hat{y}$):** Результат $\sigma(z)$ интерпретируется как **вероятность принадлежности к классу "1"** ($\hat{y} = P(y=1|X)$). Вероятность класса "0" равна $1 - \hat{y}$.
4.  **Принятие решения:** Используется порог (обычно 0.5):
    *   Если $\hat{y} \ge 0.5$, предсказывается класс "1".
    *   Если $\hat{y} < 0.5$, предсказывается класс "0".

**Свойства Сигмоиды:**

*   **Диапазон:** [0, 1].
*   **Форма:** S-образная кривая (значение 0.5 при z=0, стремится к 1 при $z \to +\infty$, к 0 при $z \to -\infty$).
*   **Дифференцируемость:** Важно для градиентного спуска. Производная: $\sigma'(z) = \sigma(z) (1 - \sigma(z))$.

```python
import numpy as np
import matplotlib.pyplot as plt

# --- Сигмоидальная функция ---
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

# --- Визуализация Сигмоиды ---
z_vals = np.linspace(-10, 10, 100)
sigmoid_vals = sigmoid(z_vals)

plt.figure(figsize=(7, 5))
plt.plot(z_vals, sigmoid_vals, label="σ(z) = 1 / (1 + e^(-z))")
plt.xlabel("z")
plt.ylabel("σ(z)")
plt.title("Сигмоидальная функция")
plt.grid(True)
plt.ylim(-0.1, 1.1) # Немного расширим диапазон Y для наглядности
plt.axhline(y=0.5, color='red', linestyle='--', label='y=0.5') # Горизонталь на 0.5
plt.axvline(x=0, color='gray', linestyle='--', label='z=0') # Вертикаль на 0
plt.legend()
plt.show()
```

---

### XI.B. Функция Потерь: Log Loss (Бинарная Кросс-Энтропия)

**Проблема MSE для Классификации:** MSE плохо подходит для вероятностей (не учитывает диапазон [0, 1], невыпуклость функции потерь при использовании с сигмоидой).

**Log Loss (Binary Cross-Entropy):** Стандартная функция потерь для бинарной классификации, измеряющая ошибку предсказания вероятностей.

**Формула (для одного объекта):**
$$L(y, \hat{y}) = - [y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]$$
*   `y`: Истинный класс (0 или 1).
*   `ŷ`: Предсказанная вероятность класса 1.
*   `log()`: Натуральный логарифм.

**Интерпретация:**
*   Если `y=1`, $L = -\log(\hat{y})$. Штраф растет, когда $\hat{y} \to 0$.
*   Если `y=0`, $L = -\log(1-\hat{y})$. Штраф растет, когда $\hat{y} \to 1$.
*   Штрафует модель за "уверенные" неправильные предсказания сильнее, чем за "неуверенные".

**Средний Log Loss (по всем N объектам):**
$$J(w, b) = - \frac{1}{N} \sum_{i=1}^{N} [y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)]$$
Именно эту функцию $J(w, b)$ мы минимизируем при обучении.

```python
# --- Функция Log Loss ---
def calculate_log_loss(y_true, y_predicted_proba):
  epsilon = 1e-15 # Для численной стабильности (избегаем log(0))
  y_predicted_proba = np.clip(y_predicted_proba, epsilon, 1 - epsilon)
  log_loss = - np.mean(y_true * np.log(y_predicted_proba) + (1 - y_true) * np.log(1 - y_predicted_proba))
  return log_loss

# --- Пример использования ---
y_true_example = np.array([1, 0, 1, 0, 1])
y_predicted_proba_example = np.array([0.9, 0.1, 0.8, 0.3, 0.7])
log_loss_example = calculate_log_loss(y_true_example, y_predicted_proba_example)
# print(f"Пример Log Loss = {log_loss_example:.4f}") # Около 0.1935
```

---

### XI.C. Обучение: Градиентный Спуск для Log Loss

**Цель:** Найти оптимальные `w` и `b`, минимизирующие средний Log Loss $J(w, b)$.

**Градиенты Log Loss:** (Удивительно похожи на градиенты MSE для линейной регрессии!)

*   **По весам `w`:**
    $$\frac{\partial J}{\partial w} = \frac{1}{N} X^T (\hat{y} - y)$$
    (где $\hat{y} = \sigma(Xw+b)$)
*   **По смещению `b`:**
    $$\frac{\partial J}{\partial b} = \frac{1}{N} \sum_{i=1}^{N} (\hat{y}_i - y_i) = \text{mean}(\hat{y} - y)$$

**Алгоритм Градиентного Спуска:**

1.  Инициализировать `w` и `b` (нулями или малыми случайными числами).
2.  Повторять `num_iterations` раз:
    *   Вычислить предсказанные вероятности: $\hat{y} = \sigma(Xw + b)$.
    *   Вычислить градиенты $\frac{\partial J}{\partial w}$ и $\frac{\partial J}{\partial b}$.
    *   Обновить параметры:
        $w := w - \eta \frac{\partial J}{\partial w}$
        $b := b - \eta \frac{\partial J}{\partial b}$
        (где $\eta$ - `learning_rate`).

```python
# --- Функции градиентов ---
def calculate_gradient_log_loss_w(X, y_true, y_predicted_proba):
  errors = y_predicted_proba - y_true.reshape(-1, 1) # Ensure y_true is column vector if needed
  gradient_w = (1 / len(y_true)) * np.dot(X.T, errors)
  return gradient_w

def calculate_gradient_log_loss_b(y_true, y_predicted_proba):
  errors = y_predicted_proba - y_true.reshape(-1, 1) # Ensure y_true is column vector if needed
  gradient_b = np.mean(errors)
  return gradient_b

# --- Реализация градиентного спуска (как в исходном коде) ---
# (Генерация данных, инициализация, цикл градиентного спуска,
#  вычисление y_predicted_proba_logistic, log_loss, градиентов, обновление w и b)
# ... (код из блоков 1, 5 исходника) ...

# --- Визуализация процесса обучения (как в исходном коде) ---
# (Построение графика log_loss_history)
# ... (код из блока 6 исходника) ...

# --- Визуализация разделяющей границы (как в исходном коде) ---
# (Функция predict_class, генерация сетки, предсказание на сетке, contourf, contour)
# ... (код из блока 7 исходника) ...
# ВАЖНО: использовать обученные w_logistic, b_logistic в этом блоке!

# --- Оценка Accuracy (как в исходном коде) ---
# (Предсказание классов на обучающих данных, вычисление accuracy_score)
# ... (код из блока 8 исходника) ...

# --- Сравнение с sklearn (как в исходном коде) ---
# (Обучение sklearn.LogisticRegression, сравнение параметров, Log Loss, Accuracy,
#  визуализация двух decision boundaries рядом)
# ... (код из блока 9 исходника) ...
```

*Примечание: Код для градиентного спуска, визуализаций и сравнения оставлен как в исходном варианте для полноты, но может быть сокращен или вынесен в функции для большей компактности реального конспекта.*

---

### XI.D. Мультиклассовая Логистическая Регрессия

**Проблема:** Бинарная логистическая регрессия работает только для двух классов.

**Решение:** Расширение на K > 2 классов с использованием функции **Softmax**.

**Функция Softmax:**
Преобразует вектор "скоров" (сырых выходов модели) $z = [z_1, ..., z_K]$ в вектор вероятностей $\hat{y} = [\hat{y}_1, ..., \hat{y}_K]$, где $\sum \hat{y}_i = 1$ и $\hat{y}_i \in [0, 1]$.

**Формула:**
$$\text{softmax}(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$
*   Экспоненты делают скоры положительными и усиливают различия.
*   Деление на сумму экспонент нормализует результат в вероятности.

**Как работает мультиклассовая логистическая регрессия:**

1.  **Линейные комбинации:** Для *каждого класса* $k$ вычисляется свой скор:
    $z_k = X w_k + b_k$
    (Теперь есть матрица весов $W$ размера `(K, n)` и вектор смещений $b$ размера `(K,)`).
2.  **Softmax:** Вектор скоров $z = [z_1, ..., z_K]$ пропускается через Softmax для получения вектора вероятностей $\hat{y} = \text{softmax}(z)$.
3.  **Предсказание:** Выбирается класс с максимальной вероятностью: $\text{argmax}_k(\hat{y}_k)$.

**Функция Потерь: Категориальная Кросс-Энтропия**
Обобщение Log Loss на K классов.

**Формула (для одного объекта с one-hot вектором `y`):**
$$L(y, \hat{y}) = - \sum_{i=1}^{K} y_i \log(\hat{y}_i)$$
*   `y = [y_1, ..., y_K]`: One-hot вектор истинного класса (1 для истинного класса, 0 для остальных).
*   $\hat{y} = [\hat{y}_1, ..., \hat{y}_K]$: Вектор предсказанных вероятностей (выход Softmax).

**Упрощение (если $c$ - индекс истинного класса):**
$$L = - \log(\hat{y}_c)$$
Минимизация этой функции заставляет модель максимизировать вероятность истинного класса.

**Средняя Категориальная Кросс-Энтропия (по N объектам):**
$$J(W, b) = - \frac{1}{N} \sum_{j=1}^{N} \sum_{i=1}^{K} y_{ij} \log(\hat{y}_{ij})$$

**Обучение:** Обычно используется готовая реализация (например, `sklearn.linear_model.LogisticRegression` с параметром `multi_class='multinomial'`). Алгоритмы оптимизации (например, `lbfgs`) минимизируют категориальную кросс-энтропию для нахождения оптимальных матриц $W$ и векторов $b$.

```python
# --- Практика Мультиклассовой Лог. Регрессии с sklearn ---
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression as SklearnLogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Генерация данных (как в исходном коде, 3 класса)
# ... (код из блока 1 мультиклассового примера) ...

# 2. Обучение модели sklearn
# Используем multi_class='multinomial' для Softmax
model_sklearn_multiclass = SklearnLogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42)
model_sklearn_multiclass.fit(X_multiclass, y_multiclass)
print("\n--- Мультиклассовая LogisticRegression (sklearn) обучена ---")
# print("Параметры w:\n", model_sklearn_multiclass.coef_) # Матрица (3, 2)
# print("Параметры b:", model_sklearn_multiclass.intercept_) # Вектор (3,)

# 3. Предсказание вероятностей и классов (как в исходном коде)
# ... (код из блока 3 мультиклассового примера) ...
# y_predicted_proba_multiclass_sklearn = model_sklearn_multiclass.predict_proba(X_multiclass)
# y_predicted_class_multiclass_sklearn = model_sklearn_multiclass.predict(X_multiclass)
# print("\nПримеры предсказанных вероятностей:\n", y_predicted_proba_multiclass_sklearn[:3])
# print("Примеры предсказанных классов:", y_predicted_class_multiclass_sklearn[:3])


# 4. Оценка качества: Accuracy и Confusion Matrix (как в исходном коде)
# ... (код из блока 4 мультиклассового примера) ...
# accuracy_multiclass = accuracy_score(y_multiclass, y_predicted_class_multiclass_sklearn)
# cm_multiclass = confusion_matrix(y_multiclass, y_predicted_class_multiclass_sklearn)
# print(f"\nAccuracy (мультикласс): {accuracy_multiclass:.4f}")
# (код для визуализации confusion matrix с seaborn)
# plt.show() # Показать все графики в конце
```

---
Этот вариант объединяет теорию и практику, убирает избыточность, но сохраняет ключевые формулы, объяснения и код для демонстрации. Графики и сравнение с `sklearn` оставлены как важные элементы для понимания.